In [1]:
import numpy as np
import pandas as pd
import os
from scipy import sparse
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
#import xgboost as xgb
import lightgbm as lgb

In [2]:
ad_feature=pd.read_csv('../data/adFeature.csv')
user_feature=pd.read_csv('../data/userFeature/userFeature_1.csv')#29
train=pd.read_csv('../data/train/train_1.csv')
predict=pd.read_csv('../data/test1/test1_1.csv')

In [3]:
#ad_feature=pd.read_csv('../data/adFeature.csv')
#user_feature = pd.read_csv('../data/userFeature.csv')
#train = pd.read_csv('../data/train.csv')
#predict = pd.read_csv('../data/test1.csv')

In [4]:
# 下面根据前面的特征分析，剔除掉缺失率过高的特征: interest3, interest4, kw3, appIdInstall, topic3
one_hot_feature=['creativeSize', 'LBS','age','carrier','consumptionAbility','education','gender','house','advertiserId','campaignId', 'creativeId',
       'adCategoryId', 'productId', 'productType']
vector_feature=['os','marriageStatus','interest1','interest2','interest5','kw1','kw2','topic1','topic2']

train.loc[train['label']==-1,'label']=0
predict['label']=-1
data = pd.concat([train,predict])
data = pd.merge(data,ad_feature,on='aid',how='left')
data = pd.merge(data,user_feature,on='uid',how='left')
data = data.fillna('-1')

print('start!')
for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])

train = data[data.label != -1]
data_clicked = train[train['label'] == 1]

start!


In [5]:
# 增加广告点击率特征
num_ad = train['aid'].value_counts().sort_index()
num_ad_clicked = data_clicked['aid'].value_counts().sort_index()

ratio = num_ad_clicked / num_ad

ratio_clicked = pd.DataFrame({
    'aid': ratio.index,
    'ratio_clicked' : ratio.values
})

data = pd.merge(data, ratio_clicked, on=['aid'], how='left')
print('加入广告点击率特征,OK!')
#print(ratio_clicked)

加入广告点击率特征,OK!


In [6]:
# 增加每个广告推送给不同的用户数
num_advertise_touser = train.groupby('aid').uid.nunique()
num_advertise_touser = pd.DataFrame({
    'aid': num_advertise_touser.index,
    'num_advertise_touser' : num_advertise_touser.values
})

data = pd.merge(data, num_advertise_touser, on=['aid'], how='left')
print('加入广告推送给不同用户数量特征,OK!')
#print(num_advertise_touser)

加入广告推送给不同用户数量特征,OK!


In [7]:
# 加入推广计划转化率
num_campaign = train['campaignId'].value_counts().sort_index()
num_campaign_clicked = data_clicked['campaignId'].value_counts().sort_index()
ratio_num_campaign = num_campaign_clicked / num_campaign
ratio_num_campaign = pd.DataFrame({
    'campaignId': ratio_num_campaign.index,
    'ratio_num_campaign' : ratio_num_campaign.values
})

data = pd.merge(data, ratio_num_campaign, on=['campaignId'], how='left')
print('加入推广计划转化率特征,OK!')
#print(ratio_num_campaign)

加入推广计划转化率特征,OK!


In [8]:
# 加入学历所对应转化率
num_education = train['education'].value_counts().sort_index()
num_education_clicked = data_clicked['education'].value_counts().sort_index()
ration_num_education = num_education_clicked / num_education
ration_num_education = pd.DataFrame({
    'education': ration_num_education.index,
    'ration_num_education' : ration_num_education.values
})

data = pd.merge(data, ration_num_education, on=['education'], how='left')
print('加入学历所对应转化率特征,OK!')
#print(ration_num_education)

加入学历所对应转化率特征,OK!


In [9]:
# 加入用户所在LBS的历史点击率
num_lbs = train.groupby('LBS').uid.nunique()
num_lbs_clicked = data_clicked.groupby('LBS').uid.nunique()
ratio_num_lbs = num_lbs_clicked  / num_lbs 
ratio_num_lbs = ratio_num_lbs.fillna(0)
ratio_num_lbs = pd.DataFrame({
    'LBS': ratio_num_lbs.index,
    'ration_num_LBS' : ratio_num_lbs.values
})

data = pd.merge(data, ratio_num_lbs, on=['LBS'], how='left')
print('加入用户所在LBS的历史点击率特征,OK!')
#print(ratio_num_lbs)

加入用户所在LBS的历史点击率特征,OK!


In [10]:
#加入用户兴趣总数
vec = data[['interest1', 'interest2', 'interest5', 'kw1', 'kw2', 'topic1', 'topic2']]
num_vec = []
train_array = np.array(vec)

for i in range(train_array.shape[0]):
    num = 0
    inter = train_array[i]
    for j in inter:
        inter_lis = j.split(' ')
        if inter_lis[0] == '-1':
            continue
        num += len(inter_lis)
    num_vec.append(num)
    
num_all_vec  = pd.DataFrame(num_vec, columns=['num_all_vec'])

data =  pd.concat([data, num_all_vec], axis=1)
print('加入用户兴趣总数,OK!')

加入用户兴趣总数,OK!


In [11]:
# 分离测试集
data = data.fillna(0)
train = data[data.label != -1]
test = data[data.label == -1]
res = test[['aid','uid']]
test = test.drop('label', axis=1)
train_y = train.pop('label')

In [12]:
# 处理联网类型特征
ct_train = train['ct'].values
ct_train = [m.split(' ') for m in ct_train]
ct_trains = []
for i in ct_train:
    index = [0, 0, 0, 0, 0]
    for j in i:
        index[int(j)] = 1
    ct_trains.append(index)

ct_test = test['ct'].values
ct_test = [m.split(' ') for m in ct_test]
ct_tests = []
for i in ct_test:
    index = [0, 0, 0, 0, 0]
    for j in i:
        index[int(j)] = 1
    ct_tests.append(index)

In [13]:
# 将上面新加入的特征进行归一化

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(train[['ratio_clicked', 'num_advertise_touser', 'ratio_num_campaign','ration_num_education', 'ration_num_LBS','num_all_vec']].values)
scaler.fit(test[['ratio_clicked', 'num_advertise_touser', 'ratio_num_campaign','ration_num_education', 'ration_num_LBS','num_all_vec']].values)
#,'ratio_num_user_getAd'
train_x = scaler.transform(train[['ratio_clicked', 'num_advertise_touser', 'ratio_num_campaign','ration_num_education', 'ration_num_LBS','num_all_vec']].values)
test_x = scaler.transform(test[['ratio_clicked', 'num_advertise_touser', 'ratio_num_campaign','ration_num_education', 'ration_num_LBS','num_all_vec']].values)

train_x = np.hstack((train_x, ct_trains))
test_x = np.hstack((test_x, ct_tests))

In [14]:
# 特征进行onehot处理
oc_encoder = OneHotEncoder()
for feature in one_hot_feature:
    oc_encoder.fit(data[feature].values.reshape(-1, 1))
    train_a=oc_encoder.transform(train[feature].values.reshape(-1, 1))
    test_a = oc_encoder.transform(test[feature].values.reshape(-1, 1))
    train_x = sparse.hstack((train_x, train_a))
    test_x = sparse.hstack((test_x, test_a))
print('one-hot prepared !')

one-hot prepared !


In [15]:
# 处理count特征向量
ct_encoder = CountVectorizer(ngram_range=(1, 2),token_pattern='(?u)\\b\\w+\\b')
for feature in vector_feature:
    ct_encoder.fit(data[feature])
    train_a = ct_encoder.transform(train[feature])
    test_a = ct_encoder.transform(test[feature])
    train_x = sparse.hstack((train_x, train_a))
    test_x = sparse.hstack((test_x, test_a))
print('cv prepared !')
# print('ths shape of train data:', test_x.shape)

cv prepared !


In [16]:
#sparse.save_npz('../data/model_fea_add1_train.npz', train_x)
#sparse.save_npz('../data/model_fea_add1_test.npz', test_x)

In [17]:
def LGB_predict(train_x, train_y, test_x, res):
    print("LGB test")
    clf = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,zero_as_missing=True,
        max_depth=-1, n_estimators=500, objective='binary',
        subsample=0.9, colsample_bytree=0.8, subsample_freq=1,
        learning_rate=0.1, min_child_weight=50, random_state=2018, n_jobs=-1
    )
    clf.fit(train_x, train_y, eval_set=[(train_x, train_y)], eval_metric='auc',early_stopping_rounds=100)
    res['score'] = clf.predict_proba(test_x)[:, 1]
    res['score'] = res['score'].apply(lambda x: float('%.6f' % x))
    res.to_csv('../data/submission.csv', index=False)
    return clf

model = LGB_predict(train_x,train_y,test_x,res)
#joblib.dump(model, './data/model_fea_add1.model')|

LGB test
[1]	valid_0's auc: 0.533222
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.534747
[3]	valid_0's auc: 0.536059
[4]	valid_0's auc: 0.537087
[5]	valid_0's auc: 0.537432
[6]	valid_0's auc: 0.53795
[7]	valid_0's auc: 0.538334
[8]	valid_0's auc: 0.538474
[9]	valid_0's auc: 0.538728
[10]	valid_0's auc: 0.538927
[11]	valid_0's auc: 0.53881
[12]	valid_0's auc: 0.538919
[13]	valid_0's auc: 0.539113
[14]	valid_0's auc: 0.53907
[15]	valid_0's auc: 0.539191
[16]	valid_0's auc: 0.539208
[17]	valid_0's auc: 0.539398
[18]	valid_0's auc: 0.539483
[19]	valid_0's auc: 0.539421
[20]	valid_0's auc: 0.539645
[21]	valid_0's auc: 0.539752
[22]	valid_0's auc: 0.539947
[23]	valid_0's auc: 0.539939
[24]	valid_0's auc: 0.540005
[25]	valid_0's auc: 0.540158
[26]	valid_0's auc: 0.540185
[27]	valid_0's auc: 0.540394
[28]	valid_0's auc: 0.54033
[29]	valid_0's auc: 0.54039
[30]	valid_0's auc: 0.540491
[31]	valid_0's auc: 0.540556
[32]	valid_0's auc: 0.540695
[33]	valid_0's

[277]	valid_0's auc: 0.546969
[278]	valid_0's auc: 0.546961
[279]	valid_0's auc: 0.54698
[280]	valid_0's auc: 0.546993
[281]	valid_0's auc: 0.547026
[282]	valid_0's auc: 0.547062
[283]	valid_0's auc: 0.547064
[284]	valid_0's auc: 0.547063
[285]	valid_0's auc: 0.547083
[286]	valid_0's auc: 0.547146
[287]	valid_0's auc: 0.547127
[288]	valid_0's auc: 0.547151
[289]	valid_0's auc: 0.547129
[290]	valid_0's auc: 0.547161
[291]	valid_0's auc: 0.547149
[292]	valid_0's auc: 0.547168
[293]	valid_0's auc: 0.547216
[294]	valid_0's auc: 0.547231
[295]	valid_0's auc: 0.547257
[296]	valid_0's auc: 0.547286
[297]	valid_0's auc: 0.5473
[298]	valid_0's auc: 0.547304
[299]	valid_0's auc: 0.547357
[300]	valid_0's auc: 0.547368
[301]	valid_0's auc: 0.547354
[302]	valid_0's auc: 0.547368
[303]	valid_0's auc: 0.547383
[304]	valid_0's auc: 0.547384
[305]	valid_0's auc: 0.547358
[306]	valid_0's auc: 0.547383
[307]	valid_0's auc: 0.547397
[308]	valid_0's auc: 0.547388
[309]	valid_0's auc: 0.547384
[310]	valid_0

C:\Users\caijiayue\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:447: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')
